# Deploy ControlNet models to SageMaker endpoint

Prepare model artifacts

In [12]:
import tarfile
from pathlib import Path
import tqdm
from time import gmtime, strftime
import datetime
import boto3
import json

import sagemaker
from sagemaker import get_execution_role
from sagemaker.s3 import S3Uploader, s3_path_join

from sagemaker import Model
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

session = sagemaker.Session()
bucket = session.default_bucket()
prefix = "sagemaker/generative-ai-controlnet"
role = get_execution_role()

account = session.boto_session.client("sts").get_caller_identity()["Account"]
region = session.boto_session.region_name

In [2]:
def create_tar(tarfile_name: str, local_path: Path):
    """
    Create a tar.gz archive with the content of `local_path`.
    """
    file_list = [k for k in local_path.glob("**/*.*") if f"{k.relative_to(local_path)}"[0] != "."]
    pbar = tqdm.tqdm(file_list, unit="files")
    with tarfile.open(tarfile_name, mode="w:gz") as archive:
        for k in pbar:
            pbar.set_description(f"{k}")
            archive.add(k, arcname=f"{k.relative_to(local_path)}")
    tar_size = Path(tarfile_name).stat().st_size / 10**6
    return 

To create the tarball file, the below code might take more than 5mins due to the model size is quite big. Please be patient.

In [ ]:
%%time
tar_file = "model.tar.gz"
tar_size = create_tar(tar_file, Path("./ControlNet/models"))
print(f"Created {tar_file}")

In [ ]:
model_data_path = s3_path_join("s3://", bucket, prefix + "/models")
print(f"Uploading Models to {model_data_path}")
model_uri = S3Uploader.upload("model.tar.gz", model_data_path)
print(f"Uploaded roberta model to {model_uri}")

In [9]:
now = f"{datetime.datetime.now():%Y-%m-%d-%H-%M-%S}"
model_name = f"sagemaker-gai-demo-{now}"
model = Model(
    image_uri="631450739534.dkr.ecr.us-east-1.amazonaws.com/sagemaker-gai-controlnet-hosting:latest", #replace with your own model_uri
    model_data=model_uri,
    name=model_name, 
    role=role,
)
print(model_name)

In [10]:
endpoint_name = model_name

model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.xlarge',
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
    endpoint_name=endpoint_name
)

-----------!

In [13]:
file_name = "./data.json"
with open(file_name, 'r') as file:
    json_data = json.load(file)
    
sm_runtime = boto3.client("sagemaker-runtime")

In [14]:
%%time
response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(json_data),
    ContentType="application/json",
)
data = response["Body"].read().decode('utf-8')

CPU times: user 44.6 ms, sys: 23.6 ms, total: 68.3 ms
Wall time: 15.4 s


In [15]:
output = json.loads(data)
detected_map = output["detected_map"]
print(type(detected_map))
image = output["image"]
print(type(image))

<class 'list'>
<class 'list'>
